In [2]:
import requests
import csv
from datetime import datetime

# Define the API URL for Mannheim (City ID = 195)
url = "https://api.nextbike.net/maps/nextbike-live.json?city=195"

# Fetch the data from the API
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

    # Extract station data
    stations = data.get('countries', [])[0].get('cities', [])[0].get('places', [])

    # Get the current timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Specify the CSV file name
    csv_file = "mannheim_nextbike_data.csv"

    # Open the CSV file for writing
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write the header row
        writer.writerow([
            "Station ID", "Name", "Latitude", "Longitude", "Available Bikes", "Free Slots", "Is Renting", "Is Returning", "Timestamp"
        ])

        # Write station details
        for station in stations:
            writer.writerow([
                station.get('uid'),
                station.get('name'),
                station.get('lat'),
                station.get('lng'),
                station.get('bikes', 0),
                station.get('free_racks', 0),
                station.get('is_renting'),
                station.get('is_returning'),
                timestamp
            ])

    print(f"Data saved to {csv_file}")
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")


Data saved to mannheim_nextbike_data.csv


In [6]:

import requests
import csv
from datetime import datetime
import time
from threading import Timer

# Define the API URL for Mannheim (City ID = 195)
url = "https://api.nextbike.net/maps/nextbike-live.json?city=195"

def fetch_and_save_data():
    # Fetch the data from the API
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()

        # Extract station data
        stations = data.get('countries', [])[0].get('cities', [])[0].get('places', [])

        # Get the current timestamp
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Specify the CSV file name
        csv_file = "mannheim_nextbike_data.csv"

        # Open the CSV file for writing
        with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)

            # Write the header row if the file is empty
            if file.tell() == 0:
                writer.writerow([
                    "Station ID", "Name", "Latitude", "Longitude", "Available Bikes", "Free Slots", "Is Renting", "Is Returning", "Timestamp"
                ])

            # Write station details
            for station in stations:
                writer.writerow([
                    station.get('uid'),
                    station.get('name'),
                    station.get('lat'),
                    station.get('lng'),
                    station.get('bikes', 0),
                    station.get('free_racks', 0),
                    station.get('is_renting'),
                    station.get('is_returning'),
                    timestamp
                ])

        print(f"Data saved to {csv_file} at {timestamp}")
    else:
        print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")

# Define the interval in seconds (e.g., every 15 minutes)
interval = 900  # 15 minutes

def schedule_task():
    fetch_and_save_data()
    Timer(interval, schedule_task).start()

# Start the scheduler
schedule_task()


Data saved to mannheim_nextbike_data.csv at 2024-11-27 18:17:50


In [9]:
import requests
import csv
from datetime import datetime
import time
from threading import Timer

# Define the GBFS auto-discovery URL for Capital Bikeshare
gbfs_url = "https://gbfs.capitalbikeshare.com/gbfs/2.3/gbfs.json"

def fetch_gbfs_data():
    try:
        # Fetch the GBFS auto-discovery data
        response = requests.get(gbfs_url)
        response.raise_for_status()
        gbfs_data = response.json()

        # Extract the station information and station status URLs
        en_feeds = gbfs_data['data']['en']['feeds']
        station_info_url = next(feed['url'] for feed in en_feeds if feed['name'] == 'station_information')
        station_status_url = next(feed['url'] for feed in en_feeds if feed['name'] == 'station_status')

        # Fetch station information
        station_info_response = requests.get(station_info_url)
        station_info_response.raise_for_status()
        station_info = station_info_response.json()

        # Fetch station status
        station_status_response = requests.get(station_status_url)
        station_status_response.raise_for_status()
        station_status = station_status_response.json()

        # Merge station information and status
        stations = {station['station_id']: station for station in station_info['data']['stations']}
        for status in station_status['data']['stations']:
            station_id = status['station_id']
            if station_id in stations:
                stations[station_id].update(status)

        return stations.values()

    except requests.RequestException as e:
        print(f"Error fetching GBFS data: {e}")
        return []

def save_to_csv(stations):
    # Get the current timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Specify the CSV file name
    csv_file = "capital_bikeshare_data.csv"

    # Open the CSV file for appending
    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write the header row if the file is empty
        if file.tell() == 0:
            writer.writerow([
                "Station ID", "Name", "Latitude", "Longitude", "Capacity",
                "Bikes Available", "Docks Available", "Is Renting", "Is Returning", "Last Reported", "Timestamp"
            ])

        # Write station details
        for station in stations:
            writer.writerow([
                station.get('station_id'),
                station.get('name'),
                station.get('lat'),
                station.get('lon'),
                station.get('capacity'),
                station.get('num_bikes_available'),
                station.get('num_docks_available'),
                station.get('is_renting'),
                station.get('is_returning'),
                datetime.fromtimestamp(station.get('last_reported')).strftime("%Y-%m-%d %H:%M:%S"),
                timestamp
            ])

    print(f"Data saved to {csv_file} at {timestamp}")

def scheduled_task():
    stations = fetch_gbfs_data()
    if stations:
        save_to_csv(stations)
    else:
        print("No data to save.")
    Timer(interval, scheduled_task).start()

# Define the interval in seconds (e.g., every 15 minutes)
interval = 900  # 15 minutes

# Start the scheduler
scheduled_task()


Data saved to capital_bikeshare_data.csv at 2024-11-27 19:21:48
